## Procesamiento de datasets

In [2]:
%load_ext lab_black

import cv2
import json
import lxml.etree as le
import numpy as np
import os
import pandas as pd
import sys

BASE_PATH = os.path.join("/tf/notebooks/CEIA/computer-vision-2/tp-final")
sys.path.insert(0, BASE_PATH)

from utils import load_image, get_filename_generator

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Para realizar la detección de patentes se utilizarán los siguientes datasets.

- [Artificial mercosur license plates](https://data.mendeley.com/datasets/nx9xbs4rgx/2)
- [Car license plate detection - Kaggle](https://www.kaggle.com/datasets/andrewmvd/car-plate-detection)
- [Motorcycle License Plate Dataset](https://www.youtube.com/watch?v=B-30SXOo5Yw)
- [License plate detection - Roboflow](https://universe.roboflow.com/thesis-yhihj/license-plate-detection-wiomm)
- [License plate image - Roboflow](https://universe.roboflow.com/class-3icb6/license-plate-wu0bx)

[Link](https://drive.google.com/drive/folders/1GEuvq0Y3SAJNUqWJ4MmKXDq40Kq0FHpw?usp=sharing) a todos los datasets.

Los distintos datasets utilizan distintas convenciones para almacenar las coordenadas de los bounding boxes.
Se va a procesar la información de todos los archivos para crear un único dataset unificado que posea el formato necesario para entrenar los modelos utilizar.

Durante el procesamiento también se van a descartar los datos que no son necesarios, se van a asignar nuevos nombres a las imagenes y almacenarlas utilizando una estrucra de directorios adecuada.


In [3]:
SEED = 123

BASE_DS_PATH = "/tf/datasets"

FINAL_DS_PATH = os.path.join(BASE_DS_PATH, "final-dataset")

try:
    original_umask = os.umask(0)
    os.mkdir(FINAL_DS_PATH, mode=0o775)
except FileExistsError:
    pass
finally:
    os.umask(original_umask)


DS_PATHS = {
    "mercosur": os.path.join(BASE_DS_PATH, "artificial-mercosur-license-plates"),
    "kaggle": os.path.join(BASE_DS_PATH, "kaggle-license-plate-detection"),
    "motorcycle": os.path.join(BASE_DS_PATH, "motorcycle-license-plate"),
    "roboflow1": os.path.join(BASE_DS_PATH, "roboflow-license-plate-detection"),
    "roboflow2": os.path.join(BASE_DS_PATH, "roboflow-license-plate-image"),
}

filename_generator = get_filename_generator()
files_annotations = []

### Preprocesamiento de [Artificial mercosur license plates](https://data.mendeley.com/datasets/nx9xbs4rgx/2)

In [4]:
MERCOSUR_PATHS = {
    "dataset": os.path.join(DS_PATHS["mercosur"], "dataset.csv"),
    "labels": os.path.join(DS_PATHS["mercosur"], "labels"),
    "images": os.path.join(DS_PATHS["mercosur"], "images"),
}

mercosur_df = pd.read_csv(MERCOSUR_PATHS["dataset"], index_col=0)
mercosur_df

,image,label,class,x_center,y_center,width,height
0,monitoring_system_4.JPG,monitoring_system_4.txt,0,0.180625,0.650833,0.128750,0.058333
1,monitoring_system_1817.JPG,monitoring_system_1817.txt,0,0.747500,0.034167,0.072500,0.031667
2,monitoring_system_1864.JPG,monitoring_system_1864.txt,0,0.714375,0.152500,0.096250,0.041667
3,monitoring_system_2484.JPG,monitoring_system_2484.txt,0,0.632500,0.625833,0.127500,0.058333
4,monitoring_system_2066.JPG,monitoring_system_2066.txt,0,0.885000,0.085000,0.087500,0.040000
...,...,...,...,...,...,...,...
3835,monitoring_system_2793.JPG,monitoring_system_2793.txt,0,0.616875,0.830833,0.163750,0.071667
3836,monitoring_system_264.JPG,monitoring_system_264.txt,0,0.801250,0.102500,0.102500,0.038333
3837,monitoring_system_2718.JPG,monitoring_system_2718.txt,0,0.925625,0.087500,0.083750,0.035000
3838,parking_lot1_35.JPG,parking_lot1_35.txt,0,0.388940,0.544118,0.057904,0.046841


Muchas imágenes tienen más de un vehiculo, esto es un caso muy poco probable en el caso a resolver, por lo tanto de descartan estas imágenes.

Inspeccionando el dataset se observa que hay imágenes repetidas con diferentes tipos de zoom. Como los casos con vehiculos muy lejanos a la cámara son poco probables se descartan.

In [5]:
results_counter = {"successful": 0, "errors": 0}

for i in range(len(mercosur_df)):
    try:
        image_filename = mercosur_df.iloc[i]["image"]
        image_label = mercosur_df.iloc[i]["label"]
        new_filename = next(filename_generator)

        # Open the image
        img = load_image(MERCOSUR_PATHS["images"], image_filename)
        (height, width, _) = img.shape

        # Get box info
        box_width = int(mercosur_df.iloc[i]["width"] * width)
        box_height = int(mercosur_df.iloc[i]["height"] * height)
        x_center = int(mercosur_df.iloc[i]["x_center"] * width)
        x_min = int(x_center - box_width / 2)
        x_max = int(x_center + box_width / 2)
        y_center = int(mercosur_df.iloc[i]["y_center"] * height)
        y_min = int(y_center - box_height / 2)
        y_max = int(y_center + box_height / 2)

        files_annotations.append(
            (
                MERCOSUR_PATHS["images"],
                image_filename,
                new_filename,
                width,
                height,
                int(x_min),
                int(y_min),
                int(x_max),
                int(y_max),
            )
        )

        results_counter["successful"] += 1
    except:
        results_counter["errors"] += 1

print(f"{results_counter['successful']} processed successfully")
print(f"{results_counter['errors']} errors")

[ WARN:0@28.546] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/tf/datasets/artificial-mercosur-license-plates/images/parking_lot1_188.JPG'): can't open/read file: check file path/integrity
[ WARN:0@48.468] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/tf/datasets/artificial-mercosur-license-plates/images/parking_lot1_98.JPG'): can't open/read file: check file path/integrity
[ WARN:0@70.651] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/tf/datasets/artificial-mercosur-license-plates/images/parking_lot1_37.JPG'): can't open/read file: check file path/integrity
[ WARN:0@73.878] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/tf/datasets/artificial-mercosur-license-plates/images/parking_lot1_99.JPG'): can't open/read file: check file path/integrity
[ WARN:0@85.609] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/tf/datase

3832 processed successfully
8 errors


### Preprocesamiento de [Car license plate detection - Kaggle](https://www.kaggle.com/datasets/andrewmvd/car-plate-detection)

La información de los bounding boxes vienen dada en formato xml. Extraemos los datos necesarios y los almacenamos en un vector.

In [6]:
KAGGLE_PATHS = {
    "annotations": os.path.join(DS_PATHS["kaggle"], "annotations"),
    "images": os.path.join(DS_PATHS["kaggle"], "images"),
}

In [7]:
elements_delete_list = ["folder", "pose", "truncated", "occluded", "difficult"]
results_counter = {"successful": 0, "errors": 0}

for image_filename in os.listdir(KAGGLE_PATHS["images"]):
    try:
        # xml filename
        image_label = image_filename[:-4] + ".xml"

        new_filename = next(filename_generator)

        # Open the image
        img = load_image(KAGGLE_PATHS["images"], image_filename)
        (height, width, _) = img.shape

        # Get info from the xml file
        tree = le.parse(os.path.join(KAGGLE_PATHS["annotations"], image_label))
        for element_name in elements_delete_list:
            for elem in tree.xpath(".//" + element_name):
                parent = elem.getparent()
                parent.remove(elem)
        for elem in tree.xpath(".//filename"):
            elem.text = new_filename
        for elem in tree.xpath(".//name"):
            elem.text = "license"

        x_min = [elem.text for elem in tree.xpath(".//xmin")][-1]
        y_min = [elem.text for elem in tree.xpath(".//ymin")][-1]
        x_max = [elem.text for elem in tree.xpath(".//xmax")][-1]
        y_max = [elem.text for elem in tree.xpath(".//ymax")][-1]

        files_annotations.append(
            (
                KAGGLE_PATHS["images"],
                image_filename,
                new_filename,
                width,
                height,
                int(x_min),
                int(y_min),
                int(x_max),
                int(y_max),
            )
        )

        results_counter["successful"] += 1

    except:
        results_counter["errors"] += 1

print(f"{results_counter['successful']} processed successfully")
print(f"{results_counter['errors']} errors")

433 processed successfully
0 errors


### Preprocesamiento de [Motorcycle License Plate Dataset](https://www.youtube.com/watch?v=B-30SXOo5Yw)

In [8]:
MOTORCYCLE_PATHS = {
    "annotations": os.path.join(DS_PATHS["motorcycle"], "annotations"),
    "images": os.path.join(DS_PATHS["motorcycle"], "images"),
}

Estas etiquetas vienen dadas en formato xml, por lo tanto, la lógica a utilizar es similar a la utilizada en el dataset anterior.

In [9]:
elements_delete_list = ["folder", "path", "source", "pose", "truncated", "difficult"]
results_counter = {"successful": 0, "errors": 0}

for image_filename in os.listdir(MOTORCYCLE_PATHS["images"]):
    try:
        # xml filename
        image_label = image_filename[:-4] + ".xml"

        new_filename = next(filename_generator)

        # Open the image
        img = load_image(MOTORCYCLE_PATHS["images"], image_filename)
        (height, width, _) = img.shape

        # Edit original xml file
        tree = le.parse(os.path.join(MOTORCYCLE_PATHS["annotations"], image_label))
        for element_name in elements_delete_list:
            for elem in tree.xpath(".//" + element_name):
                parent = elem.getparent()
                parent.remove(elem)
        for elem in tree.xpath(".//filename"):
            elem.text = new_filename
        for elem in tree.xpath(".//name"):
            elem.text = "license"

        x_min = [elem.text for elem in tree.xpath(".//xmin")][-1]
        y_min = [elem.text for elem in tree.xpath(".//ymin")][-1]
        x_max = [elem.text for elem in tree.xpath(".//xmax")][-1]
        y_max = [elem.text for elem in tree.xpath(".//ymax")][-1]

        files_annotations.append(
            (
                MOTORCYCLE_PATHS["images"],
                image_filename,
                new_filename,
                width,
                height,
                int(x_min),
                int(y_min),
                int(x_max),
                int(y_max),
            )
        )

        results_counter["successful"] += 1

    except:
        results_counter["errors"] += 1

print(f"{results_counter['successful']} processed successfully")
print(f"{results_counter['errors']} errors")

1591 processed successfully
0 errors


### Preprocesamiento de [License plate detection - Roboflow](https://universe.roboflow.com/thesis-yhihj/license-plate-detection-wiomm)

Roboflow permite descargar los datasets con diferentes formatos. Este dataset se encuentra en el formato csv utilizado por `Tensorflow Object Detection`. Se eligió descargarlo en este formato porque es la fácilidad de trabajarlo.

In [10]:
TRAIN_PATH = os.path.join(DS_PATHS["roboflow1"], "train")
TEST_PATH = os.path.join(DS_PATHS["roboflow1"], "test")
VALID_PATH = os.path.join(DS_PATHS["roboflow1"], "valid")

ROBOFLOW1_PATHS = {
    "train": {
        "images": TRAIN_PATH,
        "dataset": os.path.join(TRAIN_PATH, "_annotations.csv"),
    },
    "test": {
        "images": TEST_PATH,
        "dataset": os.path.join(TEST_PATH, "_annotations.csv"),
    },
    "valid": {
        "images": VALID_PATH,
        "dataset": os.path.join(VALID_PATH, "_annotations.csv"),
    },
}

roboflow1_train_df = pd.read_csv(ROBOFLOW1_PATHS["train"]["dataset"])
roboflow1_train_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,1934_png.rf.9505f7b156d8a5f955fdb23a1560328d.jpg,313,79,0,92,4,158,20
1,2457_png.rf.94d223ec7ddc29b4e3ab7ea013eff1c6.jpg,336,106,0,196,24,269,51
2,2755_png.rf.952e8a8557c3ba459d8833badc731991.jpg,374,275,0,184,168,267,195
3,1699_png.rf.9477889e9495140b81d30efb4e556f7b.jpg,239,83,0,94,0,155,21
4,6107_png.rf.944ccebaf09866546cc755f5c9311459.jpg,41,29,0,15,17,25,23
...,...,...,...,...,...,...,...,...
4565,3176_png.rf.36459d6ea6b5e919dcb0dd2290a54b3f.jpg,2086,807,0,524,538,1326,752
4566,6389_png.rf.36a4337be572c5c6b60e99b75b918fc9.jpg,775,435,0,244,263,521,329
4567,2461_png.rf.340dfdb37297ddab83b1577641241abc.jpg,279,142,0,189,60,256,84
4568,5321_png.rf.0c3339f236c8fe1abd2df4733d0f8761.jpg,231,341,0,6,247,96,275


In [11]:
results_counter = {"successful": 0, "errors": 0}

for folder in ["train", "test", "valid"]:
    df = pd.read_csv(ROBOFLOW1_PATHS[folder]["dataset"])

    for i in range(len(df)):
        try:
            (
                image_filename,
                width,
                height,
                _,
                x_min,
                y_min,
                x_max,
                y_max,
            ) = df.iloc[i]

            new_filename = next(filename_generator)

            # Open the image
            img = load_image(ROBOFLOW1_PATHS[folder]["images"], image_filename)

            files_annotations.append(
                (
                    ROBOFLOW1_PATHS[folder]["images"],
                    image_filename,
                    new_filename,
                    width,
                    height,
                    int(x_min),
                    int(y_min),
                    int(x_max),
                    int(y_max),
                )
            )

            results_counter["successful"] += 1
        except:
            results_counter["errors"] += 1

print(f"{results_counter['successful']} processed successfully")
print(f"{results_counter['errors']} errors")

6532 processed successfully
0 errors


### Preprocesamiento de [License plate image - Roboflow](https://universe.roboflow.com/class-3icb6/license-plate-wu0bx)

El dataset también tiene el formato de `Tensorflow Object Detection` asi que el procesamiento es idéntico al caso anterior.

In [12]:
TRAIN_PATH = os.path.join(DS_PATHS["roboflow2"], "train")
TEST_PATH = os.path.join(DS_PATHS["roboflow2"], "test")
VALID_PATH = os.path.join(DS_PATHS["roboflow2"], "valid")

ROBOFLOW2_PATHS = {
    "train": {
        "images": TRAIN_PATH,
        "dataset": os.path.join(TRAIN_PATH, "_annotations.csv"),
    },
    "test": {
        "images": TEST_PATH,
        "dataset": os.path.join(TEST_PATH, "_annotations.csv"),
    },
    "valid": {
        "images": VALID_PATH,
        "dataset": os.path.join(VALID_PATH, "_annotations.csv"),
    },
}

roboflow2_train_df = pd.read_csv(ROBOFLOW2_PATHS["train"]["dataset"])
roboflow2_train_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,96_jpg.rf.9141a9821db9905eda13051948fd8a19.jpg,416,416,0,79,196,230,255
1,594_jpg.rf.90d9515bf42866fbe32cdca20e45375a.jpg,416,416,0,136,222,260,278
2,711_jpg.rf.9245330cc2f1503aa85b3105d81b3c0e.jpg,416,416,0,148,207,261,246
3,1116_jpg.rf.954dc2c3dcc52469a0bdfed3a25b23b5.jpg,416,416,0,25,205,60,240
4,403_jpg.rf.9448b788b6fbd4ab81e3f9173822e0c4.jpg,416,416,0,94,244,262,329
...,...,...,...,...,...,...,...,...
1616,529_jpg.rf.65629483e273ef1bce13da94da1e81d2.jpg,416,416,0,125,220,262,282
1617,623_jpg.rf.66db5597ba1188cdd73c5f780b0f471f.jpg,416,416,0,117,228,249,279
1618,1030_jpg.rf.673feff1e48530de8cef7e48daf4835e.jpg,416,416,0,254,209,299,251
1619,1030_jpg.rf.673feff1e48530de8cef7e48daf4835e.jpg,416,416,0,78,250,114,289


In [13]:
results_counter = {"successful": 0, "errors": 0}

for folder in ["train", "test", "valid"]:
    df = pd.read_csv(ROBOFLOW2_PATHS[folder]["dataset"])

    for i in range(len(df)):
        try:
            (
                image_filename,
                width,
                height,
                _,
                x_min,
                y_min,
                x_max,
                y_max,
            ) = df.iloc[i]

            new_filename = next(filename_generator)

            # Open the image
            img = load_image(ROBOFLOW2_PATHS[folder]["images"], image_filename)

            files_annotations.append(
                (
                    ROBOFLOW2_PATHS[folder]["images"],
                    image_filename,
                    new_filename,
                    width,
                    height,
                    int(x_min),
                    int(y_min),
                    int(x_max),
                    int(y_max),
                )
            )

            results_counter["successful"] += 1
        except:
            results_counter["errors"] += 1

print(f"{results_counter['successful']} processed successfully")
print(f"{results_counter['errors']} errors")

1965 processed successfully
0 errors


### Generar datasets de train, test y validation

Con toda la información recolectada de los diferentes dataset creamos un DataFrame de pandas que se utilizará para almacenar el nuevo dataset en el formato que necesiten los modelos a entrenar.

In [14]:
columns = [
    "path",
    "original_filename",
    "new_filename",
    "width",
    "height",
    "xmin",
    "ymin",
    "xmax",
    "ymax",
]

df = pd.DataFrame(files_annotations, columns=columns)
df

,path,original_filename,new_filename,width,height,xmin,ymin,xmax,ymax
0,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_4.JPG,00001.jpg,800,600,92,373,195,407
1,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_1817.JPG,00002.jpg,800,600,569,10,626,29
2,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_1864.JPG,00003.jpg,800,600,532,78,609,103
3,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_2484.JPG,00004.jpg,800,600,454,358,556,392
4,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_2066.JPG,00005.jpg,800,600,673,39,743,63
...,...,...,...,...,...,...,...,...,...
14348,/tf/datasets/roboflow-license-plate-image/valid,319_jpg.rf.4634675ac94f18bb58fa1c33227affec.jpg,14357.jpg,416,416,125,235,277,299
14349,/tf/datasets/roboflow-license-plate-image/valid,778_jpg.rf.499f36bd32e79b7e69fce470d678055a.jpg,14358.jpg,416,416,102,251,274,317
14350,/tf/datasets/roboflow-license-plate-image/valid,156_jpg.rf.46bd87bf6a2992e0ff2b487cc6d63645.jpg,14359.jpg,416,416,131,253,291,306
14351,/tf/datasets/roboflow-license-plate-image/valid,958_jpg.rf.498e0cf431968c725a125f6ce35031af.jpg,14360.jpg,416,416,131,262,248,319


Se divide el dataset en `train`, `validation` y `test` con un split de 80/10/10.

In [15]:
indices = [idx for idx in df.index]
np.random.seed(SEED)
np.random.shuffle(indices)

train_end = int(0.85 * len(df))
val_end = int(0.95 * len(df))

train_indices = indices[:train_end]
val_indices = indices[train_end:val_end]
test_indices = indices[val_end:]

splitted_df = {
    "train": df.iloc[train_indices],
    "val": df.iloc[val_indices],
    "test": df.iloc[test_indices],
}

print(f"train size: {len(train_indices)}")
print(f"validation size: {len(val_indices)}")
print(f"test size: {len(test_indices)}")

splitted_df["train"]

train size: 12200
validation size: 1435
test size: 718


,path,original_filename,new_filename,width,height,xmin,ymin,xmax,ymax
5961,/tf/datasets/roboflow-license-plate-detection/...,577_png.rf.9a58d9771d1f681de0fd0f712dc4c84c.jpg,05970.jpg,291,239,97,151,181,177
5856,/tf/datasets/roboflow-license-plate-detection/...,1934_png.rf.9505f7b156d8a5f955fdb23a1560328d.jpg,05865.jpg,313,79,92,4,158,20
12355,/tf/datasets/roboflow-license-plate-detection/...,1466_png.rf.5c5451b2c5323ce0a6084d8c7072643e.jpg,12364.jpg,436,376,98,204,143,233
1482,/tf/datasets/artificial-mercosur-license-plate...,monitoring_system_2583.JPG,01487.jpg,800,600,61,134,144,161
5698,/tf/datasets/motorcycle-license-plate/images,1630979263.jpg,05707.jpg,1920,1080,822,194,1108,450
...,...,...,...,...,...,...,...,...,...
13543,/tf/datasets/roboflow-license-plate-image/train,737_jpg.rf.3d727f8ea09a89ffef142453f9c10186.jpg,13552.jpg,416,416,130,261,285,329
7688,/tf/datasets/roboflow-license-plate-detection/...,5012_png.rf.3890174448cd7c185fe0471faabc5215.jpg,07697.jpg,231,114,83,41,153,64
2194,/tf/datasets/artificial-mercosur-license-plate...,parking_lot1_189.JPG,02202.jpg,3264,1836,369,778,688,906
4970,/tf/datasets/motorcycle-license-plate/images,1630983028.jpg,04979.jpg,1920,1080,927,452,1200,693


Se genera el dataset con el formato utilizado por YOLO v7

Formato: `(<class_id>, <x_center>,  <y_center>,  <width>,  <height>)`

Estructura de archivos: 

```
yolov7-dataset
├── data.yaml
├── test
│   ├── images
│   └── labels
├── train
│   ├── images
│   └── labels
└── valid
    ├── images
    └── labels
```

In [16]:
YOLO_V7_BASE_PATH = os.path.join(FINAL_DS_PATH, "yolov7-dataset")

YOLO_V7_DATASET_PATHS = {
    "train": {
        "images": os.path.join(YOLO_V7_BASE_PATH, "train/images"),
        "labels": os.path.join(YOLO_V7_BASE_PATH, "train/labels"),
    },
    "test": {
        "images": os.path.join(YOLO_V7_BASE_PATH, "test/images"),
        "labels": os.path.join(YOLO_V7_BASE_PATH, "test/labels"),
    },
    "val": {
        "images": os.path.join(YOLO_V7_BASE_PATH, "valid/images"),
        "labels": os.path.join(YOLO_V7_BASE_PATH, "valid/labels"),
    },
}

try:
    original_umask = os.umask(0)
    for subset in YOLO_V7_DATASET_PATHS.values():
        for folder in subset.values():
            os.makedirs(folder, mode=0o775)
except FileExistsError:
    pass
finally:
    os.umask(original_umask)

In [17]:
for subset in YOLO_V7_DATASET_PATHS.keys():
    for row in splitted_df[subset].iterrows():
        (_, data) = row
        (
            path,
            original_filename,
            new_filename,
            width,
            height,
            x_min,
            y_min,
            x_max,
            y_max,
        ) = data

        img = load_image(path, original_filename)
        cv2.imwrite(
            os.path.join(YOLO_V7_DATASET_PATHS[subset]["images"], new_filename), img
        )

        # Get format used by yolov7
        x_min /= width
        x_max /= width
        x_center = x_min + (x_max - x_min) / 2
        y_min /= height
        y_max /= height
        y_center = y_min + (y_max - y_min) / 2

        txt_filename = new_filename[:-4] + ".txt"
        txt_path = os.path.join(YOLO_V7_DATASET_PATHS[subset]["labels"], txt_filename)
        with open(txt_path, "w") as f:
            f.write(f"0 {x_center} {y_center} {x_max - x_min} {y_max - y_min}")

# Create .yaml file
with open(os.path.join(YOLO_V7_BASE_PATH, "data.yaml"), "w") as f:
    train_path = YOLO_V7_DATASET_PATHS["train"]["images"]
    val_path = YOLO_V7_DATASET_PATHS["val"]["images"]
    test_path = YOLO_V7_DATASET_PATHS["test"]["images"]
    f.write(f"train: {train_path}\n")
    f.write(f"val: {val_path}\n")
    f.write(f"test: {test_path}\n\n")
    f.write("nc: 1\n")
    f.write("names: ['0']")

Se genera el dataset con el formato COCO, utilizado por Detectron2.

Formato:

```
{
  ...
  "categories": [
    {
      "id": 1,
      "name": "license_plate",
      "supercategory": ""
    }
  ],
  "images": [
    {
      "id": <id>,
      "file_name": <filename>,
      "height": <height>,
      "width": <width>,
      "license": 0,
      "flickr_url": "",
      "coco_url": "",
      "date_captured": 0
    },
    ...
  ],
  "annotations": [
    {
      "id": <id>,
      "image_id": <id>,
      "category_id": 0,
      "bbox": [<x_min>, <y_min>, <width>, <height>],
      "area": <width> * <height>,
      "segmentation": [],
      "iscrowd": 0,
      "attributes": {"occluded": False}
    },
    ...
  ]
}
```

Estructura de archivos: 

```
detectron2-dataset
├── train
├── val
└── test
```

In [18]:
DETECTRON_2_BASE_PATH = os.path.join(FINAL_DS_PATH, "detectron2-dataset")

DETECTRON_2_DATASET_PATHS = {
    "train": os.path.join(DETECTRON_2_BASE_PATH, "train"),
    "val": os.path.join(DETECTRON_2_BASE_PATH, "val"),
    "test": os.path.join(DETECTRON_2_BASE_PATH, "test"),
}

try:
    original_umask = os.umask(0)
    for folder in DETECTRON_2_DATASET_PATHS.values():
        os.makedirs(folder, mode=0o775)
except FileExistsError:
    pass
finally:
    os.umask(original_umask)

In [19]:
categories = [{"id": 1, "name": "license_plate", "supercategory": ""}]
licenses = [{"name": "", "id": 0, "url": ""}]
info = {
    "contributor": "",
    "date_created": "",
    "description": "",
    "url": "",
    "version": "",
    "year": "",
}


for subset in DETECTRON_2_DATASET_PATHS.keys():
    images = []
    annotations = []
    for row in splitted_df[subset].iterrows():
        (id, data) = row
        (
            path,
            original_filename,
            new_filename,
            width,
            height,
            x_min,
            y_min,
            x_max,
            y_max,
        ) = data

        box_width = x_max - x_min
        box_height = y_max - y_min

        images.append(
            {
                "id": id,
                "file_name": new_filename,
                "height": height,
                "width": width,
                "license": 0,
                "flickr_url": "",
                "coco_url": "",
                "date_captured": 0,
            }
        )
        annotations.append(
            {
                "id": id,
                "image_id": id,
                "category_id": 1,
                "bbox": [x_min, y_min, box_width, box_height],
                "area": box_width * box_height,
                "segmentation": [],
                "iscrowd": 0,
                "attributes": {"occluded": False},
            }
        )

        img = load_image(path, original_filename)
        cv2.imwrite(os.path.join(DETECTRON_2_DATASET_PATHS[subset], new_filename), img)

    detectron_annotations = {
        "categories": categories,
        "licenses": licenses,
        "info": info,
        "images": images,
        "annotations": annotations,
    }

    with open(
        os.path.join(DETECTRON_2_DATASET_PATHS[subset], "_annotations.coco.json"), "w"
    ) as f:
        json.dump(detectron_annotations, f)